In [42]:
import numpy as np
import yfinance as yf
from scipy.optimize import minimize

tickers = [
    "AAPL", "MSFT", "NVDA",   # Tech
    "XOM", "CVX",             # Énergie
    "TSLA",
    "BTC-USD", "ETH-USD"      # Crypto
]

data = yf.download(tickers, period="3y", interval="1d",
                   auto_adjust=True, progress=False)

# Close propre, colonnes vides retirées
prices = data["Close"].dropna(axis=1, how="all").ffill()

# Rendements
returns = prices.pct_change(fill_method=None).dropna()

expected_returns = returns.mean()
cov_matrix = returns.cov()

def portfolio_performance(weights, expected_returns, cov_matrix):
    returns = np.dot(weights, expected_returns)
    std_dev = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    return returns, std_dev

def portfolio_volatility(weights, cov_matrix):
    return np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

In [44]:
target_return = 0.001  # Target daily return

# Constraints and bounds
num_assets = len(tickers)

constraints = (
    {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1},  # Sum of weights is 1
    {'type': 'ineq', 'fun': lambda weights: np.dot(weights, expected_returns) - target_return}  # Expected return >= target return
)

max_weight = 1.0
bounds = tuple((0, max_weight) for asset in range(num_assets))

initial_guess = num_assets * [1. / num_assets]

result = minimize(portfolio_volatility, initial_guess, args=(cov_matrix,), 
                  method='SLSQP', bounds=bounds, constraints=constraints)

optimal_weights = result.x
p_return, p_std_dev = portfolio_performance(optimal_weights, expected_returns, cov_matrix)
sharpe_ratio = (p_return - 0.001) / p_std_dev if p_std_dev != 0 else np.nan

# weights ordered by size
sorted_indices = np.argsort(optimal_weights)[::-1]
print("Optimal Weights:") 
for idx in sorted_indices:
    print(f"{tickers[idx]}: {optimal_weights[idx]*100:.2f}%")


print("\nExpected Portfolio Return:",f"{p_return*100:.4f}%")
print("\nExpected Portfolio Volatility:", f"{p_std_dev*100:.4f}%")
print("\nSharpe Ratio:", f"{sharpe_ratio:.4f}")


Optimal Weights:
ETH-USD: 33.73%
CVX: 31.68%
MSFT: 14.91%
TSLA: 13.56%
NVDA: 6.13%
BTC-USD: 0.00%
XOM: 0.00%
AAPL: 0.00%

Expected Portfolio Return: 0.1000%

Expected Portfolio Volatility: 1.0233%

Sharpe Ratio: 0.0000
